In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from tqdm.cli import tqdm
from glob import glob
from sklearn.manifold import TSNE
from os import path as pathlib,walk

%matplotlib inline

In [12]:
class Dataset(object):
    """
    Dataset wrapper to read images from folder and structure it.
    """
    def __init__(self,path,n_faces,resize=128):
        self.path = pathlib.abspath(path)
        self.resize = resize

    def __repr__(self,):
        return f"Dataset Parser @ {self.path}"
        
    def parse(self,progress_bar=True):
        w = walk(self.path)
        self.data = np.array([[[r,pathlib.join(r,f_)] for f_ in f] for r,_,f in list(w)[1:]]).reshape(-1,2)
        self.y = np.array(self.data[:,0])
        self.x = np.array([
            cv2.cvtColor(
                cv2.resize(
                    cv2.imread(
                        i
                    ),
                    (self.resize,self.resize)
                ),
                cv2.COLOR_BGR2RGB
            ) / 255
            for 
                i
            in 
                (tqdm(self.data[:,1]) if progress_bar else self.data[:,1])
        ]).astype(np.float32)
        return self
                

In [13]:
dataset = Dataset("./images/train/",100)
dataset.parse(progress_bar=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 1408.24it/s]


Dataset Parser @ C:\workspace\facedet\notebooks\images\train

In [14]:
im_in = tf.keras.layers.Input( shape=( 128, 128, 3) )

net = tf.keras.applications.MobileNet(include_top=False, weights='imagenet', input_tensor=im_in)

out = tf.keras.layers.GlobalAveragePooling2D()(net.output)
out = tf.keras.layers.experimental.RandomFourierFeatures(output_dim=64, scale=64, kernel_initializer='gaussian')(out)

model = tf.keras.Model( im_in, out )

In [21]:
model.save("./model/")

INFO:tensorflow:Assets written to: ./model/assets
